# PyTorch Deep Neural Network Lab

This notebook contains code used to run the PyTorch Neural Network Lab in the DSDL App.

Note: By default every time you save this notebook the stages main cells are exported into a python module which can then get invoked by Splunk's MLTK SPL commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the DSDL app for more information about this workflow.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [306]:
# This definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import time

# Custom Lion Optimizer
class Lion(optim.Optimizer):
    def __init__(self, params, lr=1e-4, betas=(0.9, 0.99), weight_decay=0.1):
        defaults = dict(lr=lr, betas=betas, weight_decay=weight_decay)
        super(Lion, self).__init__(params, defaults)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['exp_avg'] = torch.zeros_like(p.data)

                exp_avg = state['exp_avg']
                beta1, beta2 = group['betas']

                # Update momentum
                exp_avg.mul_(beta1).add_(grad, alpha=1 - beta1)

                # Compute update
                update = exp_avg.clone().mul_(beta2).add_(grad, alpha=1 - beta2).sign_()

                # Weight decay
                if group['weight_decay'] != 0:
                    update.add_(p.data, alpha=group['weight_decay'])

                # Update parameters
                p.data.add_(update, alpha=-group['lr'])

        return loss

# Define the DNN model
class SimpleDNN(nn.Module):
    def __init__(self, input_dim, num_hidden_layers, nodes_per_layer, activation_func, dropout_rate):
        super(SimpleDNN, self).__init__()
        layers = []
        
        # Input layer
        layers.append(nn.Linear(input_dim, nodes_per_layer))
        layers.append(activation_func)
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))

        # Hidden layers
        for _ in range(num_hidden_layers - 1):
            layers.append(nn.Linear(nodes_per_layer, nodes_per_layer))
            layers.append(activation_func)
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))

        # Output layer
        layers.append(nn.Linear(nodes_per_layer, 2))  # Binary classification
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

# Global constants
nodes_per_layer = 128
batch_size = 256
class_weight = .3
num_epochs = 10
MODEL_DIRECTORY = "/srv/app/model/data/"

In [307]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.26.4
pandas version: 2.2.3


To stage the data with correct default parameters use this search:

| inputlookup dnn_lab_split.csv
| fit MLTKContainer mode=stage num_hidden_layers=3 activation_name="Tanh" dropout_rate=0.0 learning_rate=0.0001 optimizer_name="RMSprop" algo=dnn_lab * into app:dnn_lab

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("dnn_lab" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [308]:
# This cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [309]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("dnn_lab")

In [310]:
df

,SS_bytes,SS_duration,SS_inter_packet_time,SS_jitter,SS_loss,SS_packets,SS_rate,SS_service_connections,SS_time_to_live,is_train,label
0,-0.048395,-0.219041,-0.159467,-0.095410,-0.074079,-0.139108,0.251117,-0.249675,0.921018,1,1
1,-0.031063,-0.212911,-0.159388,-0.094396,0.009384,0.238575,-0.479776,0.344239,-1.118512,1,0
2,-0.048640,-0.218918,-0.159468,-0.095410,-0.074079,-0.139108,-0.478174,0.542210,-1.118512,1,0
3,-0.048949,-0.219041,-0.159467,-0.095410,-0.074079,-0.139108,0.251117,-0.051704,0.921018,1,0
4,-0.048395,-0.219041,-0.159467,-0.095410,-0.074079,-0.139108,0.345019,0.146267,0.921018,1,1
...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.039970,-0.198063,-0.158608,-0.084458,-0.002540,-0.024659,-0.498151,0.740181,-1.118512,0,0
9996,-0.041206,-0.219042,-0.159467,-0.095410,-0.074079,-0.139108,0.852092,-0.447646,0.921018,0,0
9997,-0.048704,-0.218896,-0.159467,-0.095410,-0.074079,-0.139108,-0.481425,-0.843588,-1.118512,0,0
9998,-0.049049,4.992334,4.335346,-0.095206,-0.074079,-0.139108,-0.500101,-0.843588,-1.402034,0,0


In [311]:
param

{'options': {'params': {'mode': 'stage',
   'num_hidden_layers': '3',
   'activation_name': '"Tanh"',
   'dropout_rate': '0.0',
   'learning_rate': '0.0001',
   'optimizer_name': '"RMSprop"',
   'algo': 'dnn_lab'},
  'args': ['*'],
  'feature_variables': ['*'],
  'model_name': 'dnn_lab',
  'algo_name': 'MLTKContainer',
  'mlspl_limits': {'disabled': False,
   'handle_new_cat': 'default',
   'max_distinct_cat_values': '100',
   'max_distinct_cat_values_for_classifiers': '100',
   'max_distinct_cat_values_for_scoring': '100',
   'max_fit_time': '6000',
   'max_inputs': '1000000',
   'max_memory_usage_mb': '16000',
   'max_model_size_mb': '3000',
   'max_score_time': '6000',
   'use_sampling': '1'},
  'kfold_cv': None},
 'feature_variables': ['SS_bytes',
  'SS_duration',
  'SS_inter_packet_time',
  'SS_jitter',
  'SS_loss',
  'SS_packets',
  'SS_rate',
  'SS_service_connections',
  'SS_time_to_live',
  'is_train',
  'label']}

## Stage 2 - create and initialize a model

In [312]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    input_dim = len(df.columns)-2 #remove training and flag field in input dimensionality
    num_hidden_layers = int(param['options']['params']['num_hidden_layers'])
    activation_name = param['options']['params']['activation_name'].strip('\"')
    
    # Map activation functions
    activation_mapping = {
        'ReLU': nn.ReLU(),
        'GELU': nn.GELU(),
        'Tanh': nn.Tanh()
    }
    activation_func = activation_mapping[activation_name]
    dropout_rate = float(param['options']['params']['dropout_rate'])
    
    # Convert to PyTorch tensors
    device = torch.device('cpu')
    model['num_hidden_layers'] = num_hidden_layers
    model['input_dim'] = input_dim
    model['nodes_per_layer'] = nodes_per_layer
    model['activation_name'] = activation_name
    model['dropout_rate'] = dropout_rate
    model['dnn'] = SimpleDNN(input_dim, num_hidden_layers, nodes_per_layer, activation_func, dropout_rate).to(device)
    return model

In [313]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

{'num_hidden_layers': 3, 'input_dim': 9, 'nodes_per_layer': 128, 'activation_name': 'Tanh', 'dropout_rate': 0.0, 'dnn': SimpleDNN(
  (layers): Sequential(
    (0): Linear(in_features=9, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): Tanh()
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): Tanh()
    (6): Linear(in_features=128, out_features=2, bias=True)
  )
)}


## Stage 3 - fit the model

In [314]:
# Train your model
# Returns a fit info json object and may modify the model object
def fit(model,df,param):
    summary_list = {}
    df_train = df[df['is_train'] == 1]
    df_test = df[df['is_train'] == 0]
    X_train = df_train.drop('label', axis=1).drop('is_train', axis=1)
    X_test = df_test.drop('label', axis=1).drop('is_train', axis=1)
    y_train = df_train['label']
    y_test = df_test['label']
    
    print("\nShape of X_train:", X_train.shape)
    print("Shape of X_test:", X_test.shape)
    print("Shape of y_train:", y_train.shape)
    print("Shape of y_test:", y_test.shape)

    device = torch.device('cpu')
    
    # Convert pandas Series to NumPy arrays before creating tensors
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device) # Convert Series to numpy array
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)   # Convert Series to numpy array
    
    # Create TensorDataset and DataLoader
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)

    learning_rate = float(param['options']['params']['learning_rate'])
    optimizer_name = param['options']['params']['optimizer_name'].strip('\"')

    # Calculate class weights
    total_samples = len(y_train)
    num_class_0 = np.sum(y_train == 0)
    num_class_1 = np.sum(y_train == 1)
    weight_for_class_0 = total_samples / (2.0 * num_class_0)
    weight_for_class_1 = (total_samples / (2.0 * num_class_1)) * class_weight
    class_weights = torch.tensor([weight_for_class_0, weight_for_class_1], dtype=torch.float32).to(device)

    # Create DataLoader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Define loss function with class weights
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    
    # Define optimizer
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model['dnn'].parameters(), lr=learning_rate)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model['dnn'].parameters(), lr=learning_rate)
    elif optimizer_name == 'Lion':
        optimizer = Lion(model['dnn'].parameters(), lr=learning_rate, betas=(0.9, 0.99), weight_decay=0.1)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")

    # Training loop
    print(f"\nTraining with: Layers={model['num_hidden_layers']}, Nodes={model['nodes_per_layer']}, LR={learning_rate}, "
          f"Batch={batch_size}, Epochs={num_epochs}, Dropout={model['dropout_rate']}, Activation={model['activation_name']}, "
          f"Optimizer={optimizer_name}, Class Weight={class_weight}")
    summary_list['training_settings'] = f"\nTraining with: Layers={model['num_hidden_layers']}, Nodes={model['nodes_per_layer']}, LR={learning_rate}, Batch={batch_size}, Epochs={num_epochs}, Dropout={model['dropout_rate']}, Activation={model['activation_name']}, Optimizer={optimizer_name}, Class Weight={class_weight}"
    start_time = time.time()
    model['dnn'].train()
    loss_list = []
    epoch_list = []
    for epoch in range(num_epochs):
        epoch_list.append(epoch+1)
        epoch_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model['dnn'](inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model['dnn'].parameters(), max_norm=1.0)
            optimizer.step()
            epoch_loss += loss.item() * inputs.size(0)
    
        epoch_loss = epoch_loss / len(train_dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
        loss_list.append(round(epoch_loss,4))
    
    training_time = time.time() - start_time
    print(f"Training completed in {training_time:.2f} seconds")
    summary_list['training_time'] = f"Training completed in {training_time:.2f} seconds"
    summary_list['epoch_number'] = epoch_list
    summary_list['loss_list'] = loss_list
    summary_list['final_loss'] = round(epoch_loss,4)
    with open(MODEL_DIRECTORY + "dnn_lab_loss.json", 'w') as file:
        json.dump(summary_list, file)
    
    return model

In [315]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))


Shape of X_train: (8000, 9)
Shape of X_test: (2000, 9)
Shape of y_train: (8000,)
Shape of y_test: (2000,)

Training with: Layers=3, Nodes=128, LR=0.0001, Batch=256, Epochs=10, Dropout=0.0, Activation=Tanh, Optimizer=RMSprop, Class Weight=0.3
Epoch 1/10, Loss: 0.4057
Epoch 2/10, Loss: 0.3230
Epoch 3/10, Loss: 0.3167
Epoch 4/10, Loss: 0.3132
Epoch 5/10, Loss: 0.3110
Epoch 6/10, Loss: 0.3087
Epoch 7/10, Loss: 0.3063
Epoch 8/10, Loss: 0.3034
Epoch 9/10, Loss: 0.3004
Epoch 10/10, Loss: 0.2971
Training completed in 1.13 seconds
{'num_hidden_layers': 3, 'input_dim': 9, 'nodes_per_layer': 128, 'activation_name': 'Tanh', 'dropout_rate': 0.0, 'dnn': SimpleDNN(
  (layers): Sequential(
    (0): Linear(in_features=9, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): Tanh()
    (4): Linear(in_features=128, out_features=128, bias=True)
    (5): Tanh()
    (6): Linear(in_features=128, out_features=2, bias=True)
  )
)}


## Stage 4 - apply the model

In [316]:
# Apply your model
# Returns the calculated results
def apply(model,df,param):
    try:
        X = df.drop('label', axis=1)
    except:
        X = df
    try:
        X = X.drop('is_train', axis=1) 
    except:
        X = df
    try:
        device = torch.device('cpu')
        X_tensor = torch.tensor(X.values, dtype=torch.float32).to(device)
        model['dnn'].eval()
        predictions = []
        with torch.no_grad(): 
            for i in range(X_tensor.shape[0]): 
                inputs = X_tensor[i:i+1] 
                output = model['dnn'](inputs)
                _, predicted = torch.max(output, 1)
                predictions.append(predicted.tolist()[0])   
        cols = {"Result": predictions}
        result = pd.DataFrame(data=cols)
    except Exception as e:
        cols = {"Error in Inference": [str(model)]}
        result = pd.DataFrame(data=cols)
    return result

In [317]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(apply(model,df[:100],param))

    Result
0        1
1        0
2        0
3        1
4        1
..     ...
95       0
96       1
97       1
98       0
99       0

[100 rows x 1 columns]


## Stage 5 - save the model

In [318]:
# Save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    model_path = MODEL_DIRECTORY + name + ".pth"
    torch.save(model['dnn'], model_path)
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        model_files = model.copy()
        model_files.pop('dnn', None)
        json.dump(model_files, file)
    return model

In [319]:
save(model,"dnn_lab")

{'num_hidden_layers': 3,
 'input_dim': 9,
 'nodes_per_layer': 128,
 'activation_name': 'Tanh',
 'dropout_rate': 0.0,
 'dnn': SimpleDNN(
   (layers): Sequential(
     (0): Linear(in_features=9, out_features=128, bias=True)
     (1): Tanh()
     (2): Linear(in_features=128, out_features=128, bias=True)
     (3): Tanh()
     (4): Linear(in_features=128, out_features=128, bias=True)
     (5): Tanh()
     (6): Linear(in_features=128, out_features=2, bias=True)
   )
 )}

## Stage 6 - load the model

In [320]:
# Load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model_params = json.load(file)

    input_dim = model_params['input_dim']
    num_hidden_layers = int(model_params['num_hidden_layers'])
    activation_name = model_params['activation_name']
    # Map activation functions
    activation_mapping = {
        'ReLU': nn.ReLU(),
        'GELU': nn.GELU(),
        'Tanh': nn.Tanh()
    }
    activation_func = activation_mapping[activation_name]
    dropout_rate = float(model_params['dropout_rate'])
    device = torch.device('cpu')
    nodes_per_layer = model_params['nodes_per_layer']

    model['dnn'] = SimpleDNN(input_dim, num_hidden_layers, nodes_per_layer, activation_func, dropout_rate).to(device)
    model_path = MODEL_DIRECTORY + name + ".pth"
    model['dnn'] = torch.load(model_path, weights_only=False)
    return model

In [321]:
model = load("dnn_lab")

In [322]:
model

{'dnn': SimpleDNN(
   (layers): Sequential(
     (0): Linear(in_features=9, out_features=128, bias=True)
     (1): Tanh()
     (2): Linear(in_features=128, out_features=128, bias=True)
     (3): Tanh()
     (4): Linear(in_features=128, out_features=128, bias=True)
     (5): Tanh()
     (6): Linear(in_features=128, out_features=2, bias=True)
   )
 )}

## Stage 7 - provide a summary of the model

In [323]:
# Return a model summary
def summary(model=None):
    try:
        with open(MODEL_DIRECTORY + "dnn_lab_loss.json", 'r') as file:
            loss_info = json.load(file)
    except:
        loss_info = {'training_settings': "None", 'training_time': "None", 'epoch_number':"None", 'loss_list':"None", 'final_loss': "None"}
    returns = loss_info
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code